# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [1]:
import ib_insync
print(ib_insync.__all__)

['util', 'Object', 'ContractDetails', 'ContractDescription', 'ComboLeg', 'UnderComp', 'DeltaNeutralContract', 'OrderComboLeg', 'OrderState', 'ScannerSubscription', 'SoftDollarTier', 'Execution', 'CommissionReport', 'ExecutionFilter', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'TickAttrib', 'NewsProvider', 'DepthMktDataDescription', 'PnL', 'PnLSingle', 'AccountValue', 'RealTimeBar', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'MktDepthData', 'DOMLevel', 'BracketOrder', 'TradeLogEntry', 'ScanData', 'TagValue', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'OrderCondition', 'ExecutionCondition', 'OperatorCondition', 'MarginCondition', 'ContractCondition', 'TimeCondition', 'PriceCondition', 'PercentChangeCondition', 'VolumeCondition',

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_insync import *
util.startLoop()

*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10)

<IB connected to 127.0.0.1:7497 clientId=10>

If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [4]:
ib.positions()

[Position(account='DU772802', contract=Contract(secType='STK', conId=9579970, symbol='IWM', exchange='ARCA', currency='USD', localSymbol='IWM', tradingClass='IWM'), position=210.0, avgCost=159.05424285),
 Position(account='DU772802', contract=Contract(secType='FUT', conId=269745169, symbol='ES', lastTradeDateOrContractMonth='20180615', multiplier='50', currency='USD', localSymbol='ESM8', tradingClass='ES'), position=-5.0, avgCost=135347.95),
 Position(account='DU772802', contract=Contract(secType='FOP', conId=280107184, symbol='ES', lastTradeDateOrContractMonth='20180615', strike=2700.0, right='C', multiplier='50', currency='USD', localSymbol='ESM8 C2700', tradingClass='ES'), position=3.0, avgCost=2029.19776665),
 Position(account='DU772802', contract=Contract(secType='OPT', conId=300772895, symbol='VIX', lastTradeDateOrContractMonth='20180619', strike=17.0, right='C', multiplier='100', currency='USD', localSymbol='VIX   180620C00017000', tradingClass='VIX'), position=8.0, avgCost=231.

Or filter the account values to get the liquidation value:

In [5]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU772802', tag='NetLiquidationByCurrency', value='842526', currency='BASE', modelCode='')]

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [6]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [7]:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='STK', conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,CVGXALGO,NYSENAT,PSX', priceMagnifier=1, longName='TESLA INC', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST5EDT', tradingHours='20180601:0400-20180601:2000;20180602:CLOSED;20180603:CLOSED;20180604:0400-20180604:200

### Current state vs request

Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So it is preferable to use the current state methods over requests. For example, use ``ib.openOrders()`` in preference over ``ib.reqOpenOrders()``, or ``ib.positions()`` over ``ib.reqPositions()``, etc:

In [8]:
%time l = ib.positions()

CPU times: user 16 µs, sys: 2 µs, total: 18 µs
Wall time: 24.1 µs


In [9]:
%time l = ib.reqPositions()

CPU times: user 3.23 ms, sys: 180 µs, total: 3.41 ms
Wall time: 43.8 ms


### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [10]:
util.logToConsole()

To see all debug messages (including network traffic):

In [11]:
import logging
util.logToConsole(logging.DEBUG)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [12]:
ib.disconnect()

2018-06-01 12:33:09,966 ib_insync.ib INFO Disconnecting from 127.0.0.1:7497, 153 B sent in 8 messages, 22.1 kB received in 406 messages, session time 830 ms.
2018-06-01 12:33:09,970 ib_insync.client INFO Disconnected
